In [1]:
#default_exp cli
from nbdev import *
# %nbdev_default_export cli

# cli

> This is a cli module for RemoteConnect

In [ ]:
#hide
from nbdev.showdoc import *

In [29]:
#export
from fastcore.script import call_parse, Param, bool_arg
from fastcore.basics import in_colab
from RemoteConnect.core import RemoteCode, RemoteJupyter, RemotePluto, mount_drive, connect_to_ngrok
import math
import pandas as pd
import logging
import pathlib

In [30]:
#export
app_logger = logging.getLogger('cli'); app_logger

<Logger cli (DEBUG)>

## Start Code

In [31]:
#export
@call_parse
def start_code(port:Param("Port to Start Code", type=int)=10000, 
               password:Param("Password to Start Code", type=str)=None, 
               tunnel:Param("Tunel Type", type=str)='ngrok',
               authtoken:Param("Tunnel Authtoken for ngrok", type=str)=None):
    "Starts Code Server"
    if in_colab(): mount_drive()
    remote = RemoteCode(password=password, port=port, tunnel=tunnel, authtoken=authtoken)
    remote.launch()

## Start Jupyter

In [32]:
#export
@call_parse
def start_jupyter(port:Param("Port to Start Jupyter", type=int)=9000, 
                  ui:Param("Interface to start", type=str)='notebook',
                  tunnel:Param("Tunel Type", type=str)='ngrok',
                  authtoken:Param("Tunnel Authtoken for ngrok", type=str)=None,
                  subdomain:Param("Add a subdomain only for localtunnel", type=str)=None,
                  install_code:Param("Flag to install code", type=bool)=False,
                  install_julia:Param("Flag to install code", type=bool)=False,
                  ):
    "Starts Jupyter"
    if in_colab(): 
        print("Mounting Drive")
        mount_drive()
    remote = RemoteJupyter(port=port, 
                           ui=ui, 
                           tunnel=tunnel, 
                           authtoken=authtoken,
                           subdomain=subdomain,
                           install_code=install_code, 
                           install_julia=install_julia
                          )
    remote.launch()

In [33]:
# start_jupyter(tunnel="telebit")

## Start Pluto

In [34]:
#export
@call_parse
def start_pluto(port:Param("Port to Start Jupyter", type=int)=9000,
                tunnel:Param("Tunel Type", type=str)='ngrok',
                authtoken:Param("Tunnel Authtoken for ngrok", type=str)=None):
    "Starts Pluto.jl reactive notebook"
    if in_colab(): mount_drive()
    remote = RemotePluto(port=port, tunnel=tunnel, authtoken=authtoken)
    remote.launch()

In [35]:
#export
def _convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

In [36]:
_convert_size(10240000)

'9.77 MB'

In [37]:
#export
@call_parse
def fsize(fldr:Param("Pathstring of folder to measure size")="."):
    path = pathlib.Path(fldr).resolve()
    app_logger.info(f"Current folder: {path}")

    with pd.option_context('display.max_rows', None,
                           'display.max_columns', None,
                           'display.width', 10000,
                           'display.precision', 3,
                           'display.colheader_justify', 'left'):
        df = pd.DataFrame([{"Name":p.name, "Size":_convert_size(p.stat().st_size), "Bytes":p.stat().st_size} for p in path.ls()])
        app_logger.info(f"Current foldersize: {_convert_size(df['Bytes'].sum())}")
        print(df.sort_values(by='Bytes').drop('Bytes', axis=1).reset_index(drop=True))